In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import sklearn
from nltk.corpus import stopwords
from collections import Counter
import nltk
%matplotlib inline

In [2]:
yelp = pd.read_table('amazon_cells_labelled.txt', delimiter='\t')
yelp.columns = ['sentence','sentiment']
boolyelp = pd.DataFrame()
boolyelp = yelp.sentiment.astype(bool)

yelp['sentence'] = yelp['sentence'].str.lower()
yelp['sentence'] = yelp['sentence'].str.replace(',', '')
yelp['sentence'] = yelp['sentence'].str.replace('/', '')
yelp['sentence'] = yelp['sentence'].str.replace(':', '')
yelp['sentence'] = yelp['sentence'].str.replace(';', '')
yelp['sentence'] = yelp['sentence'].str.replace("'", '')
yelp['sentence'] = yelp['sentence'].str.replace('"', '')
yelp['sentence'] = yelp['sentence'].str.replace('.', '')
yelp['sentence'] = yelp['sentence'].str.replace('-', '')
yelp['sentence'] = yelp['sentence'].str.replace('_', '')
yelp['sentence'] = yelp['sentence'].str.replace('+', '')
yelp['sentence'] = yelp['sentence'].str.replace('&', '')
yelp['sentence'] = yelp['sentence'].str.replace('$', '')
yelp['sentence'] = yelp['sentence'].str.replace('%', '')
yelp['sentence'] = yelp['sentence'].str.replace('@', '')
yelp['sentence'] = yelp['sentence'].str.replace('!', '')
yelp['sentence'] = yelp['sentence'].str.replace('(', '')
yelp['sentence'] = yelp['sentence'].str.replace(')', '')
yelp['sentence'] = yelp['sentence'].str.replace('\\', '')
yelp['sentence'] = yelp['sentence'].str.replace('*', '')
yelp['sentence'] = yelp['sentence'].str.replace('\]', '')
yelp['sentence'] = yelp['sentence'].str.replace('\[', '')
yelp = yelp.rename(columns = {'fit':'fit_feature'})
yelp = yelp.rename(columns = {'score':'score_feature'})
boolyelp.head()
#yelp.value_counts

0     True
1     True
2    False
3     True
4    False
Name: sentiment, dtype: bool

In [3]:
#keywords = ['good','great','excellent','incredible','love','recommend','pretty','useful','quality','reliable']
#raw_words = str(yelp.sentence.str.strip(' ').value_counts())
#raw_words = raw_words.lower().strip(' ')
pos_sms = yelp.loc[(yelp['sentiment'] == 1), ['sentence','sentiment']].reset_index()
del pos_sms['index']
neg_sms = yelp.loc[(yelp['sentiment'] == 0), ['sentence','sentiment']].reset_index()
del neg_sms['index']

pos_rwords = pd.Series(' '.join(pos_sms['sentence']).lower().split()).value_counts()[:]
pos_rwords = pos_rwords.to_frame().reset_index()
pos_rwords.columns = ['word','pos_count']
pos_rwords['pos_res'] = 1

words = []
stopWords = set(stopwords.words('english'))
for w in pos_rwords.word:
    if w not in stopWords and w != '2':
        words.append(w)
        
pos_rwords_final = pos_rwords[pos_rwords['word'].isin(words)]
#pos_rwords_final.word = pos_rwords_final.word.replace(to_replace=['\(','\)','.',',','-'], value='', regex=True)
#pos_rwords_final.word = default_clean(pos_rwords_final.word)
print(pos_rwords_final)
for key in pos_rwords_final.word:
    yelp[str(key)] = yelp.sentence.str.contains(str(key), case=False)
    
yelp.head()

               word  pos_count  pos_res
7             great         92        1
9             phone         86        1
14             good         62        1
16            works         46        1
21          product         33        1
23          headset         31        1
24             well         31        1
25          quality         31        1
26            sound         27        1
27        excellent         26        1
29            price         25        1
32              one         23        1
34          battery         22        1
35             nice         22        1
36              use         21        1
39             best         21        1
42             love         20        1
44        recommend         20        1
45              ive         19        1
47             case         18        1
48             like         18        1
49            would         17        1
51              ear         16        1
52           really         15        1


,sentence,sentiment,great,phone,good,works,product,headset,well,quality,...,beats,print,costs,modest,alone,set,etc,turn,carry,knock
0,good case excellent value,1,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,great for the jawbone,1,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,tied to charger for conversations lasting more...,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,the mic is great,1,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,i have to jiggle the plug to get it to line up...,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
def default_clean(text):
    text = filter(lambda x: x in string.printable, text)
    bad_chars = set(["@", "+", '<br>', '<br />', '/', "'", '"', '\\',
                       '(',')', '<p>', '\\n', '<', '>', '?', '#', ',',
                       '.', '[',']', '%', '$', '&', ';', '!', ';', ':',
                       '-', "*", "_", "=", "}", "{"])
    for char in bad_chars:
        text = text.replace(char, " ")
    text = re.sub('\d+', "", text)

    return text

In [5]:
from sklearn.model_selection import train_test_split
X = yelp[pos_rwords_final.word[:70]]
Y = boolyelp
X_train, X_test, Y_train, Y_test = train_test_split(X , Y, test_size=.30)

In [39]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB
#data = yelp[pos_rwords_final.word[:10]]
#target = boolyelp
# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train[:10], Y_train[:10])

# Classify, storing the result in a new variable.
y_pred = bnb.predict(X_test)

#bnb.score(X_test, y_test)
# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    X_train.shape[0],
    (Y_test != y_pred).sum()
))

Number of mislabeled points out of a total 699 points : 147


In [7]:
#data2 = yelp[pos_rwords_final.word[:20]]
#target = boolyelp
# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train[:20], Y_train[:20])

# Classify, storing the result in a new variable.
y_pred = bnb.predict(X_test)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    X_train.shape[0],
    (Y_test != y_pred).sum()
))

Number of mislabeled points out of a total 699 points : 150


In [8]:
#data3 = yelp[pos_rwords_final.word[:30]]
#target = boolyelp
# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train[:30], Y_train[:30])

# Classify, storing the result in a new variable.
y_pred = bnb.predict(X_test)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    X_train.shape[0],
    (Y_test != y_pred).sum()
))

Number of mislabeled points out of a total 699 points : 152


In [9]:
yelp.columns

Index(['sentence', 'sentiment', 'great', 'phone', 'good', 'works', 'product',
       'headset', 'well', 'quality',
       ...
       'beats', 'print', 'costs', 'modest', 'alone', 'set', 'etc', 'turn',
       'carry', 'knock'],
      dtype='object', length=1035)

In [10]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
def get_unigram_sentence(sentence):
    return [word for word in word_tokenize(sentence.lower()) if word not in stopWords]



yelp['sentence_unigram'] = yelp['sentence'].apply(lambda x: get_unigram_sentence(x))



#yelp['sentence_bigram'] = yelp['sentence_unigram'].apply(lambda x: [i for i in ngrams(x, 2)] if x.isin(pos_rwords_final.word))


                                                                    

In [11]:
words = []
for l in yelp.sentence_unigram:
    empty_list = []
    for w in l:
        if w in list(pos_rwords_final.word):
            empty_list.append(w)
    words.append(empty_list)


In [12]:
yelp['pos_unigrams'] = words

In [33]:
yelp['pos_bigrams'] = yelp.pos_unigrams.apply(lambda x: tuple([i for i in ngrams(x, 2)]))

C:\Users\spenser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: generator 'ngrams' raised StopIteration
  """Entry point for launching an IPython kernel.


In [28]:
yelp.columns


Index(['sentence', 'sentiment', 'great', 'phone', 'good', 'works', 'product',
       'headset', 'well', 'quality',
       ...
       'modest', 'alone', 'set', 'etc', 'turn', 'carry', 'knock',
       'sentence_unigram', 'pos_unigrams', 'pos_bigrams'],
      dtype='object', length=1038)

In [35]:
s = set()



In [37]:
for i, row in yelp.iterrows():
        #print(row[1037])
        s.add(row[1037])


In [38]:
s

{(),
 (('12', 'mega'),
  ('mega', 'pixel'),
  ('pixel', 'camera'),
  ('camera', 'part'),
  ('part', 'phone'),
  ('phone', 'reasonably'),
  ('reasonably', 'good7')),
 (('3', 'months'),
  ('months', 'screen'),
  ('screen', 'went'),
  ('went', 'black')),
 (('3', 'times'),),
 (('able', 'voice'),
  ('voice', 'dialing'),
  ('dialing', 'car'),
  ('car', 'problem')),
 (('absolutely', 'great'),),
 (('absolutely', 'problem'),
  ('problem', 'headset'),
  ('headset', 'linking'),
  ('linking', '8530'),
  ('8530', 'blackberry'),
  ('blackberry', 'curve')),
 (('accompanied', 'software'),
  ('software', 'almost'),
  ('almost', 'brilliant')),
 (('actually', 'know'),
  ('know', 'call'),
  ('call', 'rather'),
  ('rather', 'important'),
  ('important', 'feature'),
  ('feature', 'phone')),
 (('additional', 'ear'), ('ear', 'gels'), ('gels', 'provided')),
 (('allows', 'date'), ('date', 'time'), ('time', 'first')),
 (('also', 'ears'),
  ('ears', 'try'),
  ('try', 'ear'),
  ('ear', 'plug'),
  ('plug', 'ear')),

In [32]:
bigrams = yelp['pos_unigrams'].apply(lambda x: tuple([i for i in ngrams(x, 2)]))
#yelp['pos_bigrams'] = yelp[yelp['pos_bigrams'] != '[]']

Counter(bigrams).most_common(100)
#pos_bigrams = pos_bigrams.to_frame()
#for key in yelp.pos_bigrams:
    #yelp[str(key)] = yelp.sentence.str.contains(str(key), case=False)
#pos_bigrams    
#bigrams

C:\Users\spenser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: generator 'ngrams' raised StopIteration
  """Entry point for launching an IPython kernel.


[((), 142),
 ((('great', 'phone'),), 8),
 ((('works', 'great'),), 4),
 ((('dont', 'money'),), 4),
 ((('dont', 'buy'), ('buy', 'product')), 3),
 ((('customer', 'service'),), 3),
 ((('audio', 'quality'),), 2),
 ((('like', 'loud'), ('loud', 'phone')), 2),
 ((('sending', 'back'),), 2),
 ((('would', 'recommend'),), 2),
 ((('dont', 'buy'),), 2),
 ((('love', 'phone'),), 2),
 ((('sound', 'quality'),), 2),
 ((('great', 'deal'),), 2),
 ((('great', 'product'), ('product', 'price')), 2),
 ((('excellent', 'product'), ('product', 'price')), 2),
 ((('happy', 'product'),), 2),
 ((('easy', 'use'),), 2),
 ((('great', 'product'),), 2),
 ((('doesnt', 'work'),), 2),
 ((('good', 'case'), ('case', 'excellent'), ('excellent', 'value')), 1),
 ((('great', 'jawbone'),), 1),
 ((('charger', 'problems'),), 1),
 ((('mic', 'great'),), 1),
 ((('plug', 'get'),
   ('get', 'line'),
   ('line', 'right'),
   ('right', 'get'),
   ('get', 'decent'),
   ('decent', 'volume')),
  1),
 ((('several', 'several'),
   ('several', 'c

In [14]:
yelp.pos_bigrams.value_counts()

AttributeError: 'DataFrame' object has no attribute 'pos_bigrams'

In [ ]:
data4 = X_train[:30] + yelp.pos_
target = boolyelp
# Instantiate our model and store it in a new variable.
bnb = BernoulliNB(alpha=0.5)

# Fit our model to the data.
bnb.fit(data3, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data3)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data3.shape[0],
    (target != y_pred).sum()
))

In [ ]:

target = boolyelp

gauss = GaussianNB()
gauss.fit(data4)

In [36]:
from sklearn.metrics import confusion_matrix


0.8515625

In [ ]:
neg_rwords = pd.Series(' '.join(neg_sms['sentence']).lower().split()).value_counts()[:]
neg_rwords = neg_rwords.to_frame().reset_index()
neg_rwords.columns = ['word','neg_count']
neg_rwords['neg_res'] = 1
neg_rwords.head()



pos_neg_rwords = pd.merge(pos_rwords, neg_rwords, on='word', how='outer')
pos_neg_rwords = pos_neg_rwords.fillna(0)
pos_neg_rwords['pos_prob'] = pos_neg_rwords.pos_count / tsen
pos_neg_rwords['neg_prob'] = pos_neg_rwords.neg_count / tsen
pos_neg_rwords['t_pos_prob'] = pos_neg_rwords['pos_prob'] * ppos
pos_neg_rwords['t_neg_prob'] = pos_neg_rwords['neg_prob'] * pneg
pos_neg_rwords['+/-'] = pos_neg_rwords['t_pos_prob'] - pos_neg_rwords['t_neg_prob']
pos_neg_rwords.head()